# Dependency Extraction via include/import directive

## Steps
1. Recurse through each file of source, use the imports to determine dependency relations, generate TAs
2. Save the extracted dependencies to a raw.ta file

In [1]:
# Step 1: Walk through project and generate mapping of package->path:
import os
src = ".\\flink-1.17.1"
package_to_path_map = {}

# walk path
for dir_path, _, file_names in os.walk(src):
    for file_name in file_names:
        if file_name.endswith(".java"):
            package = dir_path.split("\\java\\")[-1].replace("\\", ".") + "." + file_name.replace(".java", "")
            package_to_path_map[package] = dir_path + "\\" + file_name
            
print("Package Map (length & head(10))")
print(f"Size: {len(package_to_path_map)}")
for k,v in list(package_to_path_map.items())[:10]:
    print(k + " -> " + v)

Package Map (length & head(10))
Size: 13194
org.apache.flink.FlinkVersion -> .\flink-1.17.1\flink-annotations\src\main\java\org\apache\flink\FlinkVersion.java
org.apache.flink.annotation.Experimental -> .\flink-1.17.1\flink-annotations\src\main\java\org\apache\flink\annotation\Experimental.java
org.apache.flink.annotation.Internal -> .\flink-1.17.1\flink-annotations\src\main\java\org\apache\flink\annotation\Internal.java
org.apache.flink.annotation.Public -> .\flink-1.17.1\flink-annotations\src\main\java\org\apache\flink\annotation\Public.java
org.apache.flink.annotation.PublicEvolving -> .\flink-1.17.1\flink-annotations\src\main\java\org\apache\flink\annotation\PublicEvolving.java
org.apache.flink.annotation.VisibleForTesting -> .\flink-1.17.1\flink-annotations\src\main\java\org\apache\flink\annotation\VisibleForTesting.java
org.apache.flink.annotation.docs.ConfigGroup -> .\flink-1.17.1\flink-annotations\src\main\java\org\apache\flink\annotation\docs\ConfigGroup.java
org.apache.flink.

In [6]:
# Step 2: Generate dependencies
dependencies = []

def process_line(file_path, line):
  # Example: 
  # line = import org.apache.flink.api.common.functions.MapFunction;
  # first need to remove the ; at end and split by space
  dependency = line.split(" ")[1].strip().replace(";", "")

  # now we can convert the package to a path and add the tuple to our list
  dependency = package_to_path_map.get(dependency)
  if dependency is None:
    return
  
  dependencies.append((file_path[2:].replace("\\", "/"), dependency[2:].replace("\\", "/")))

def process_file(file_path):
  with open(file_path, "r", encoding="utf-8") as f:
    lines = f.readlines()
    for line in lines:
        if not line.startswith("import") or not "org.apache.flink" in line:
          continue

        if "static" in line: 
          line = line.replace(" static", "")

        process_line(file_path, line)


def generate_dependencies():
  for dir_path, _, file_names in os.walk(src):
    for file_name in file_names:
      if file_name.endswith(".java"):
        process_file(dir_path + "\\" + file_name)


generate_dependencies()
limit = 15 

# print out the dependencies to verify they are correct
for file_path, dependency in dependencies:
  print(file_path + " -> " + dependency)

  limit -= 1
  if limit == 0:
    break

print(f"Size: {len(dependencies)}")


flink-1.17.1/flink-annotations/src/main/java/org/apache/flink/FlinkVersion.java -> flink-1.17.1/flink-annotations/src/main/java/org/apache/flink/annotation/Public.java
flink-1.17.1/flink-annotations/src/main/java/org/apache/flink/annotation/docs/ConfigGroup.java -> flink-1.17.1/flink-annotations/src/main/java/org/apache/flink/annotation/Internal.java
flink-1.17.1/flink-annotations/src/main/java/org/apache/flink/annotation/docs/ConfigGroups.java -> flink-1.17.1/flink-annotations/src/main/java/org/apache/flink/annotation/Internal.java
flink-1.17.1/flink-annotations/src/main/java/org/apache/flink/annotation/docs/Documentation.java -> flink-1.17.1/flink-annotations/src/main/java/org/apache/flink/annotation/Internal.java
flink-1.17.1/flink-architecture-tests/flink-architecture-tests-production/src/main/java/org/apache/flink/architecture/ProductionCodeArchitectureBase.java -> flink-1.17.1/flink-architecture-tests/flink-architecture-tests-production/src/main/java/org/apache/flink/architecture

In [7]:
# Step 2: Generate the raw.ta file
raw_ta_output = "./source_raw_ta/import_dependencies.raw.ta"

with open(raw_ta_output, "w+") as f:
  f.write("FACT TUPLE : \n")

  unique_file_paths = set(file_path for file_path, _ in dependencies)

  # first generate all the concrete instances
  for file_path in unique_file_paths:
    f.write(f"$INSTANCE {file_path} cFile\n")

  # now add in all the dependencies
  for file_path, dependency in dependencies:
    f.write(f"cLinks {file_path} {dependency}\n")